<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor

from modeling import ModelWithQASSHead

from transformers import AutoTokenizer,squad_convert_examples_to_features,AutoConfig

from transformers.data.metrics.squad_metrics import (
    compute_predictions_logits)

import torch

from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm

In [2]:
def predict(model, tokenizer, predict_file, output_prediction_file=None,output_nbest_file=None,
            max_seq_length=512, doc_stride=128, max_query_length=64, max_answer_length=300):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    dataset, examples, features = load_and_cache_examples(tokenizer,predict_file, max_seq_length, doc_stride, max_query_length)

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=8)

    
    all_results = []
    all_nbest = []

    for batch in tqdm(eval_dataloader):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }
            seq_len = inputs["input_ids"].size(1)

            feature_indices = batch[3]

            outputs = model(**inputs)

        for i, feature_index in enumerate(feature_indices):
            eval_feature = features[feature_index.item()]
            unique_id = int(eval_feature.unique_id)

            output = [to_list(output[i]) for output in outputs]

            start_logits, end_logits = output
            result = SquadResult(unique_id, start_logits, end_logits)

            all_results.append(result)


            
    
    do_lower_case=False
    version_2_with_negative=True
    n_best_size=20
    verbose_logging=False
    null_score_diff_threshold=0.0
    output_null_log_odds_file=None

    predictions = compute_predictions_logits(
            examples,
            features,
            all_results,
            n_best_size,
            max_answer_length,
            do_lower_case,
            output_prediction_file,
            output_nbest_file,
            output_null_log_odds_file,
            verbose_logging,
            version_2_with_negative,
            null_score_diff_threshold,
            tokenizer
        )



    
    
    return {'answer' : predictions[list(predictions.keys())[0]]}

In [3]:
def load_and_cache_examples(tokenizer, predict_file, max_seq_length, doc_stride, max_query_length):
    

    processor = SquadV2Processor()
    examples = processor.get_dev_examples(None, filename=predict_file)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=max_seq_length,
        doc_stride=doc_stride,
        max_query_length=max_query_length,
        is_training=False,
        return_dataset="pt",
        threads=1
    )

    return dataset, examples, features

In [4]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [11]:
tokenizer = AutoTokenizer.from_pretrained("D:/sustin_all/checkpoint-20700",do_lower_case=False,use_fast=False)

In [6]:
config = AutoConfig.from_pretrained('D:/sustin_all/checkpoint-20700')

In [7]:
model = ModelWithQASSHead.from_pretrained('D:/sustin_all/checkpoint-20700', config=config,
                                                          replace_mask_with_question_token=True,
                                                          mask_id=0, question_token_id=5,
                                                          initialize_new_qass=True)

In [8]:
with open("sample.json", 'r') as f:
    sample = json.load(f)

In [9]:
sample

{'version': 'v2.0',
 'data': [{'title': '까뮤이앤씨',
   'paragraphs': [{'qas': [{'is_impossible': True,
       'id': 'new_stock',
       'answers': [],
       'question': '신주의 제3자 배정 [MASK].'}],
     'context': '정 관 제 1 장 총 칙 제 1 조【상 호】이 회사는 주식회사 까뮤이앤씨라 칭한다. 영문으로는 CAMUS ENGINEERING & CONSTRUCTION Inc.라 표기한다. 제 2 조【목 적】이 회사는 다음 사업을 영위함을 목적으로 한다. 1. 건설자재의 생산 및 판매업 2. 토목·건축공사업 3. 포장공사업 4. 해외건설업 5. 건설기계기구 자재의 제조 판매 및 임대업 6. 위탁판매 및 대리업 7. 부동산매매 및 임대·전대관리 및 컨설팅업 8. 주택신축매매 및 임대관리업 9. 시설물 유지관리업 및 공동주택관리업 10. 전기공사, 기계설비공사업 11. 토지개간 및 공유수면 매립업 12. 주차장업 13. 레미콘, 아스콘, 콘크리트 제품 제조판매업 14. 중장비 임대업 및 중장비부품제작 정비사업 15. 석산개발, 골재생산 및 판매업 16. 조경공사업 17. 토목, 건축, 전기, 기계설비 및 도로포장 기타 제건설공사의 설계 및 감리업 18. 특정열사용시공업 및 가스시설 시공업 19. 군, 관용물자납품 및 건설용역 군납업 20. 소방시설공사업 21. 타일 제조판매업 22. 무역업 23. 무역대리업 24. 자동차 도·소매 및 수리업 25. 자동차 부품 및 부속품 판매업 26. 오수분뇨, 쓰레기, 축산폐수, 오수정화시설, 분뇨정화조, 하수처리시설 설계 및 시공업 27. 유료도로사업 28. 체육시설업 29. 무선통신, 방송 및 응용장비 제조판매업 30. 환경(대기오염,소음·진동,수질오염등) 관련 방지시설 설계 및 시공업 31. 국내외 자원개발사업 및 원유판매업 32. 관광숙박업 및 관광객 이용시설업 33. 

In [10]:
predict(model, tokenizer, predict_file="sample.json")

100%|██████████| 10/10 [00:47<00:00,  4.80s/it]


{'answer': '긴급한 자금조달 또는 출자전환을 위하여 국내외 금융기관 또는 기관 투자자에게 신주를 발행하는 경우'}

In [50]:
list(a.keys())

['new_stock']

In [ ]:
tlqkf={'answer' : ''}

In [15]:
len(test['data'])

4815

In [120]:
processor = SquadV2Processor()

In [122]:
examples = processor.get_dev_examples(None,filename="C:/Users/user/sustinvest/sample.json")

100%|██████████| 1/1 [00:00<00:00, 333.30it/s]


In [21]:
examples = processor.get_dev_examples(None,filename="C:/Users/user/sustinvest/test_qass_all.json")

100%|██████████| 4815/4815 [00:00<00:00, 9785.25it/s] 


In [124]:
examples[0]

In [125]:
features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=512,
        doc_stride=128,
        max_query_length=32,
        is_training=False,
        return_dataset="pt",
        threads=1)

add example index and unique id: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [128]:
len(dataset)

45

In [129]:
len(features)

45

In [35]:
from torch.utils.data import DataLoader, SequentialSampler

In [130]:
eval_sampler = SequentialSampler(dataset)

In [147]:
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=1)

In [42]:
import torch

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")

In [148]:
for batch in tqdm(eval_dataloader, desc="Evaluating"):
    batch = tuple(t.to(device) for t in batch)

Evaluating: 100%|██████████| 45/45 [00:00<00:00, 11220.72it/s]


In [58]:
[item for item in dir(tokenizer) if not item.startswith('_')]

['SPECIAL_TOKENS_ATTRIBUTES',
 'add_special_tokens',
 'add_tokens',
 'added_tokens_decoder',
 'added_tokens_encoder',
 'additional_special_tokens',
 'additional_special_tokens_ids',
 'all_special_ids',
 'all_special_tokens',
 'all_special_tokens_extended',
 'as_target_tokenizer',
 'basic_tokenizer',
 'batch_decode',
 'batch_encode_plus',
 'bos_token',
 'bos_token_id',
 'build_inputs_with_special_tokens',
 'clean_up_tokenization',
 'clean_up_tokenization_spaces',
 'cls_token',
 'cls_token_id',
 'convert_ids_to_tokens',
 'convert_tokens_to_ids',
 'convert_tokens_to_string',
 'create_token_type_ids_from_sequences',
 'decode',
 'deprecation_warnings',
 'do_basic_tokenize',
 'do_lower_case',
 'encode',
 'encode_plus',
 'eos_token',
 'eos_token_id',
 'from_pretrained',
 'get_added_vocab',
 'get_special_tokens_mask',
 'get_vocab',
 'ids_to_tokens',
 'init_inputs',
 'init_kwargs',
 'is_fast',
 'mask_token',
 'mask_token_id',
 'max_len_sentences_pair',
 'max_len_single_sentence',
 'max_model_in

In [143]:
b=tokenizer.convert_ids_to_tokens(dataset[0][0].tolist())

In [144]:
tokenizer.convert_tokens_to_string(b)

'[CLS] 황금낙하산 [SEP] 정 관 제정년월일 1984년 9월 5일 1차 개정 1984년 9월 7일 2차 개정 1985년 1월 30일 3차 개정 1985년 2월 28일 4차 개정 1985년 5월 29일 5차 개정 1988년 3월 11일 6차 개정 1991년 12월 5일 7차 개정 1994년 3월 4일 8차 개정 1996년 6월 5일 9차 개정 1997년 3월 5일 10차 개정 1998년 3월 13일 11차 개정 1999년 2월 26일 12차 개정 2003년 3월 14일 13차 개정 2004년 3월 19일 14차 개정 2006년 3월 17일 15차 개정 2009년 3월 20일 16차 개정 2010년 3월 26일 17차 개정 2012년 3월 16일 18차 개정 2015년 3월 27일 19차 개정 2017년 3월 24일 20차 개정 2019년 3월 22일 21차 개정 2020년 3월 27일 22차 개정 2021년 3월 26일 23차 개정 2022년 3월 25일 24차 개정 2023년 3월 24일 제 1 장 총 칙 제 1 조 ( 상호 ) 본 회사는 주식회사 E1 이라 칭하며 , 한글로는 주식회사 이원 , 영문으로는 E1 Corporation으로 한다 . 제 2 조 ( 목적 ) 본 회사는 하기사항을 경영함을 목적으로 한다 . 1 ) 액화석유가스를 포함한 석유제품과 각종 가스 및 가스기기의 수출입 , 제조 , 저장 , 운송 및 판매업 2 ) 화공약품 매매업 ( 독극물 제외 ) 3 ) 부동산 임대업 및 부동산 개발업 4 ) 항만 운영 개발 및 항만 하역업 5 ) 항만 관련 장비 및 기기 임대업 6 ) 자동차 운송사업 및 해상 운송사업 7 ) 물류시설 운영업 , 물류 관련 서비스업 , 기타 보관 및 창고업 , 물류 관련 경영 컨설팅업 , 물류 관련 응용 소프트웨어 개발 및 공급업 , 기타 종합물류사업 8 ) 연료전지 , 석탄액화가스화 , 수소 , 태양광 , 태양열 , 바이오에너지 , 풍력 , 수력 , 해양에너지 , 폐기물에너지 , 지열에너지 등 신재생에너지와 관련된 사

In [145]:
b=tokenizer.convert_ids_to_tokens(dataset[1][0].tolist())

In [146]:
tokenizer.convert_tokens_to_string(b)

'[CLS] 황금낙하산 [SEP] 14차 개정 2006년 3월 17일 15차 개정 2009년 3월 20일 16차 개정 2010년 3월 26일 17차 개정 2012년 3월 16일 18차 개정 2015년 3월 27일 19차 개정 2017년 3월 24일 20차 개정 2019년 3월 22일 21차 개정 2020년 3월 27일 22차 개정 2021년 3월 26일 23차 개정 2022년 3월 25일 24차 개정 2023년 3월 24일 제 1 장 총 칙 제 1 조 ( 상호 ) 본 회사는 주식회사 E1 이라 칭하며 , 한글로는 주식회사 이원 , 영문으로는 E1 Corporation으로 한다 . 제 2 조 ( 목적 ) 본 회사는 하기사항을 경영함을 목적으로 한다 . 1 ) 액화석유가스를 포함한 석유제품과 각종 가스 및 가스기기의 수출입 , 제조 , 저장 , 운송 및 판매업 2 ) 화공약품 매매업 ( 독극물 제외 ) 3 ) 부동산 임대업 및 부동산 개발업 4 ) 항만 운영 개발 및 항만 하역업 5 ) 항만 관련 장비 및 기기 임대업 6 ) 자동차 운송사업 및 해상 운송사업 7 ) 물류시설 운영업 , 물류 관련 서비스업 , 기타 보관 및 창고업 , 물류 관련 경영 컨설팅업 , 물류 관련 응용 소프트웨어 개발 및 공급업 , 기타 종합물류사업 8 ) 연료전지 , 석탄액화가스화 , 수소 , 태양광 , 태양열 , 바이오에너지 , 풍력 , 수력 , 해양에너지 , 폐기물에너지 , 지열에너지 등 신재생에너지와 관련된 사업 일체 9 ) 전기통신사업중 부가통신사업 , 정보통신서비스제공사업 , 통신판매업 , 전자금융업 , 위치기반서비스업 10 ) 생활 / 공업 / 농업 등 각종 용수의 생산과 공급 , 하수 / 폐수의 이송과 처리 및 이와 연관된 사업 일체 11 ) 전기자동차충전사업과 관련된 사업 일체 12 ) 소프트웨어 및 하드웨어 개발 , 제작 , 자문 , 공급 , 판매 ( 도소매 ) , 온라인컨텐츠 개발 및 시스템 통합구축 서비스의 판매업 , 위치기반 서비스 제공 사업 13 ) 발전 , 송전 

In [167]:
b=tokenizer.convert_ids_to_tokens(batch[0][0].tolist())

In [168]:
tokenizer.convert_tokens_to_string(b)

'[CLS] 황금낙하산 [SEP]의 의견이 있을 때 나 . 감사위원 전원의 동의가 있을 때 9 . 제 8항에 따라 이사회가 승인한 경우에는 대표이사는 지체 없이 대차대조표를 공고하여야 하며 , 제 1항의 각 서류의 내용을 주주총회에 보고하여야 한다 . 제 30 조 ( 이익배당금 ) 1 . 이익의 배당은 금전 , 주식 및 기타의 재산으로 할 수 있다 . 2 . 이익배당금은 매영업년도 말일현재 주주명부에 기재된 주주 또는 등록질권자에게 지급한다 . 3 . 이익배당금은 정기주주총회에서 승인한 날로부터 1개월 이내에 지급한다 . 그러나 주주총회에서 그 지급시기를 따로 정할 수 있다 . 4 . 이익배당금의 청구권은 5년간 이를 행사하지 아니하면 소멸시효가 완성되며시효의 완성으로 인한 이익배당금은 본 회사에 귀속한다 . 5 . 유상증자 , 무상증자 및 주식배당에 의하여 신주를 발행하는 경우 , 신주에 대한 이익의 배당에 관하여는 신주를 발행한 때가 속하는 영업년도의 직전 영업년도말에 발행된 것으로 본다 . 제 30 조의 2 ( 중간배당 ) 1 . 회사는 이사회 결의로 관계 법령에 따른 중간 배당을 할 수 있다 . 2 . 중간 배당은 금전으로 한다 . 3 . 회사는 이사회 결의로 제1항의 배당을 받을 주주를 확정하기 위한 기준일을 정할 수 있으며 , 기준일을 정한 경우 그 기준일의 2주 전에 이를 공고하여야 한다 . 4 . 유상증자 , 무상증자 및 주식배당에 의하여 신주를 발행하는 경우에는 제30조 제5항을 준용한다 . 부 칙 이 정관은 제39기 정기주주총회에서 승인한 날로부터 시행한다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [176]:
b=tokenizer.convert_ids_to_tokens(batch[0][0].tolist())

In [177]:
tokenizer.convert_tokens_to_string(b)

'[CLS] 황금낙하산 [SEP]는 E1 Corporation으로 한다 . 제 2 조 ( 목적 ) 본 회사는 하기사항을 경영함을 목적으로 한다 . 1 ) 액화석유가스를 포함한 석유제품과 각종 가스 및 가스기기의 수출입 , 제조 , 저장 , 운송 및 판매업 2 ) 화공약품 매매업 ( 독극물 제외 ) 3 ) 부동산 임대업 및 부동산 개발업 4 ) 항만 운영 개발 및 항만 하역업 5 ) 항만 관련 장비 및 기기 임대업 6 ) 자동차 운송사업 및 해상 운송사업 7 ) 물류시설 운영업 , 물류 관련 서비스업 , 기타 보관 및 창고업 , 물류 관련 경영 컨설팅업 , 물류 관련 응용 소프트웨어 개발 및 공급업 , 기타 종합물류사업 8 ) 연료전지 , 석탄액화가스화 , 수소 , 태양광 , 태양열 , 바이오에너지 , 풍력 , 수력 , 해양에너지 , 폐기물에너지 , 지열에너지 등 신재생에너지와 관련된 사업 일체 9 ) 전기통신사업중 부가통신사업 , 정보통신서비스제공사업 , 통신판매업 , 전자금융업 , 위치기반서비스업 10 ) 생활 / 공업 / 농업 등 각종 용수의 생산과 공급 , 하수 / 폐수의 이송과 처리 및 이와 연관된 사업 일체 11 ) 전기자동차충전사업과 관련된 사업 일체 12 ) 소프트웨어 및 하드웨어 개발 , 제작 , 자문 , 공급 , 판매 ( 도소매 ) , 온라인컨텐츠 개발 및 시스템 통합구축 서비스의 판매업 , 위치기반 서비스 제공 사업 13 ) 발전 , 송전 , 변전 , 배전을 포함한 전력사업 및 집단에너지 사업 14 ) 발전소 및 발전시설의 국내외 건설 및 운영 등의 사업 수행 및 관련 부대사업 15 ) 브랜드 , 캐릭터 상표권 등 지식재산권을 활용한 라이선스업 16 ) 위 각호의 목적 달성에 수반 또는 관련되거나 회사에 직접 간접으로 유익한 일체의 사업 제 3 조 ( 본점 ) 본 회사의 본점은 서울특별시에 두며 본 회사 이사회의 결의에 의하여 기타 필요한 지역에 지점을 둘 수있다 . 제 4 조 ( 공고방법 ) 본 회사의 공고는 회사의 인터넷 홈페이지 ( ht

In [17]:
tokenizer.tokenize("통계학 개론")

['통계', '##학', '개', '##론']

In [73]:
inputs = {"input_ids": batch[0],"attention_mask": batch[1],"token_type_ids": batch[2]}

In [91]:
outputs = model(**inputs)

In [93]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [95]:
output = [to_list(output[0]) for output in outputs]

In [99]:
start_logits, end_logits = output

In [100]:
start_logits

[58.072731018066406,
 3.4988722801208496,
 0.24470067024230957,
 -0.3985159397125244,
 2.9057106971740723,
 0.4714045524597168,
 -0.5319309234619141,
 7.3106513023376465,
 27.047534942626953,
 9.508378982543945,
 11.464067459106445,
 10.516929626464844,
 6.269726753234863,
 6.236553192138672,
 6.647543907165527,
 9.694568634033203,
 5.958996295928955,
 -0.31757187843322754,
 6.50844144821167,
 5.702147006988525,
 8.31884479522705,
 7.024850368499756,
 9.066373825073242,
 7.885194778442383,
 0.41519641876220703,
 -3.860485076904297,
 -2.0656020641326904,
 -0.1961972713470459,
 -8.133874893188477,
 -8.931615829467773,
 -5.0835723876953125,
 -6.116320610046387,
 -2.4483914375305176,
 -6.5258378982543945,
 -4.87282133102417,
 -8.352726936340332,
 -11.697896003723145,
 -11.474336624145508,
 -6.6491899490356445,
 -3.407304286956787,
 -11.666927337646484,
 -10.016741752624512,
 -6.360171318054199,
 -9.210956573486328,
 -10.318460464477539,
 -10.857963562011719,
 -7.130500793457031,
 -9.300889

In [ ]:
predictions = compute_predictions_logits(
            examples,
            features,
            all_results,
            args.n_best_size,
            args.max_answer_length,
            args.do_lower_case,
            output_prediction_file,
            None if args.dont_output_nbest else output_nbest_file,
            output_null_log_odds_file,
            args.verbose_logging,
            args.version_2_with_negative,
            args.null_score_diff_threshold,
            tokenizer,
        )

In [106]:
len(end_logits)

512

In [103]:
result = SquadResult('0', start_logits, end_logits)

In [104]:
result

In [150]:
all_results = []

In [174]:
num=0
for batch in tqdm(eval_dataloader):
    model.eval()
    batch = tuple(t.to(device) for t in batch)
    num+=1
    if num==3:
        break

  4%|▍         | 2/45 [00:00<00:00, 798.15it/s]


In [178]:
for batch in tqdm(eval_dataloader):
    model.eval()
    batch = tuple(t.to(device) for t in batch)
        
    with torch.no_grad():
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2],
        }
        seq_len = inputs["input_ids"].size(1)


        feature_indices = batch[3]

        outputs = model(**inputs)

    for i, feature_index in enumerate(feature_indices):
        eval_feature = features[feature_index.item()]
        unique_id = int(eval_feature.unique_id)
            
        output = [to_list(output[i]) for output in outputs]

        start_logits, end_logits = output
        result = SquadResult(unique_id, start_logits, end_logits)

        all_results.append(result)

100%|██████████| 45/45 [00:32<00:00,  1.39it/s]


In [183]:
len(all_results)

45

In [221]:
max_answer_length=300
do_lower_case=False
version_2_with_negative=True
n_best_size=20
output_prediction_file='prediction_file.json'
dont_output_nbest='n_prediction_file.json'
verbose_logging=False
null_score_diff_threshold=0.0
output_null_log_odds_file=None

In [222]:
predictions = compute_predictions_logits(
            examples,
            features,
            all_results,
            n_best_size,
            max_answer_length,
            do_lower_case,
            output_prediction_file,
            dont_output_nbest,
            output_null_log_odds_file,
            verbose_logging,
            version_2_with_negative,
            null_score_diff_threshold,
            tokenizer
        )

In [223]:
predictions

OrderedDict([('gold', '긴급한 자금의 조달을 위하여 국내외 금융기관 등에게 신주를 발행하는 경우')])

In [208]:
with open("n_prediction_file.json", 'r', encoding='utf-8') as f:
    data=json.load(f)

In [220]:
data

{'gold': [{'text': '긴급한 자금의 조달을 위하여 국내외 금융기관 등에게 신주를 발행하는 경우',
   'probability': 0.9999999388490493,
   'start_logit': 47.80926513671875,
   'end_logit': 35.16165542602539},
  {'text': '자금의 조달을 위하여 국내외 금융기관 등에게 신주를 발행하는 경우',
   'probability': 4.5956049037665177e-08,
   'start_logit': 30.913684844970703,
   'end_logit': 35.16165542602539},
  {'text': '긴급',
   'probability': 1.347071883372179e-08,
   'start_logit': 47.80926513671875,
   'end_logit': 17.038908004760742},
  {'text': '경우',
   'probability': 1.0999886358125239e-09,
   'start_logit': 27.181299209594727,
   'end_logit': 35.16165542602539},
  {'text': '긴급한 자금의 조달을 위하여 국내외 금융기관 등에게',
   'probability': 5.56504276733598e-10,
   'start_logit': 47.80926513671875,
   'end_logit': 13.852309226989746},
  {'text': '조달을 위하여 국내외 금융기관 등에게 신주를 발행하는 경우',
   'probability': 2.984145022277396e-11,
   'start_logit': 23.574142456054688,
   'end_logit': 35.16165542602539},
  {'text': '긴급한',
   'probability': 1.8096043697581225e-11,
   'start_logit

In [ ]:
def evaluate(model, tokenizer, prefix=""):
    dataset, examples, features = load_and_cache_examples(args, tokenizer, evaluate=True, output_examples=True)


    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)

    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    
    
    all_results = []
    all_nbest = []
    start_time = timeit.default_timer()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }
            seq_len = inputs["input_ids"].size(1)

            if args.model_type in ["xlm", "roberta", "distilbert", "camembert"]:
                del inputs["token_type_ids"]

            feature_indices = batch[3]

           
            outputs = model(**inputs)

        for i, feature_index in enumerate(feature_indices):
            eval_feature = features[feature_index.item()]
            unique_id = int(eval_feature.unique_id)

            output = [to_list(output[i]) for output in outputs]

            
            start_logits, end_logits = output
            result = SquadResult(unique_id, start_logits, end_logits)

            all_results.append(result)

    evalTime = timeit.default_timer() - start_time
    logger.info("  Evaluation done in total %f secs (%f sec per example)", evalTime, evalTime / len(dataset))

    # Compute predictions
    output_prediction_file = os.path.join(args.output_dir, "predictions_{}.json".format(prefix))
    output_nbest_file = os.path.join(args.output_dir, "nbest_predictions_{}.json".format(prefix))

    if args.version_2_with_negative:
        output_null_log_odds_file = os.path.join(args.output_dir, "null_odds_{}.json".format(prefix))
    else:
        output_null_log_odds_file = None

    # XLNet and XLM use a more complex post-processing procedure
    if args.model_type in ["xlnet", "xlm"]:
        start_n_top = model.config.start_n_top if hasattr(model, "config") else model.module.config.start_n_top
        end_n_top = model.config.end_n_top if hasattr(model, "config") else model.module.config.end_n_top

        predictions = compute_predictions_log_probs(
            examples,
            features,
            all_results,
            args.n_best_size,
            args.max_answer_length,
            output_prediction_file,
            output_nbest_file,
            output_null_log_odds_file,
            start_n_top,
            end_n_top,
            args.version_2_with_negative,
            tokenizer,
            args.verbose_logging,
        )
    else:
        predictions = compute_predictions_logits(
            examples,
            features,
            all_results,
            args.n_best_size,
            args.max_answer_length,
            args.do_lower_case,
            output_prediction_file,
            None if args.dont_output_nbest else output_nbest_file,
            output_null_log_odds_file,
            args.verbose_logging,
            args.version_2_with_negative,
            args.null_score_diff_threshold,
            tokenizer,
        )

    # Compute the F1 and exact scores.
    results = squad_evaluate(examples, predictions)
    if args.nbest_calculation:
        with open(output_nbest_file, "r") as f:
            nbest_predictions = json.load(f)

        for n in [1, 3, 5, 10]:
            exact_scores, f1_scores = get_raw_scores_nbest(examples, nbest_predictions, n)
            nbest_eval = make_eval_dict(exact_scores, f1_scores)
            merge_eval(results, nbest_eval, f"{n}_best")

    return results